In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import adhtools.utils
import pandas as pd
import os
import numpy as np

## Load corpus

In [3]:
import glob

book_files = glob.glob('/media/sf_VBox_Shared/Arabic/Fiqh/2018-11-20-Fiqh-light10-chapters/*.xml')
fnames = [os.path.basename(fn) for fn in book_files]
print(len(book_files))

10065


In [4]:
path_to_stopwordlist = '/media/sf_VBox_Shared/Arabic/arabic_stop-words_7-8-2018.txt'
external_stopwords = [line.strip() for line in open(path_to_stopwordlist, 'r', encoding='utf-8')]

In [5]:
corpus = adhtools.utils.corpus_str(book_files, analyzer=False, field='proposed_root')

## Make document-term matrix

In [6]:
vectorizer = CountVectorizer( stop_words=external_stopwords, min_df=2, max_df=0.9)
X = vectorizer.fit_transform(corpus)

In [7]:
feature_names = vectorizer.get_feature_names()
len(feature_names)

79401

In [8]:
# some example words from the first document
np.array(feature_names)[X[0].indices][:50]

array(['اعلم', 'ابدانهم', 'حمل', 'يشق', 'سيد', 'يطيق', 'يكلف', 'وعمل',
       'اكل', 'ترفق', 'تميز', 'يحتاج', 'اذن', 'رخص', 'وج', 'كر', 'اجماع',
       'ادم', 'وخصا', 'فجااز', 'حمير', 'بغال', 'ذهب', 'ينقص', 'خيل',
       'ونه', 'لحوم', 'صلاح', 'معز', 'بخصا', 'اتخاذ', 'لعب', 'عيال',
       'وقوت', 'قو', 'لعيش', 'يصطاد', 'صيد', 'تت', 'يبيت', 'يرجع', 'يروح',
       'صحرا', 'يصحب', 'غنم', 'ماش', 'جل', 'زرع', 'حراس', 'عقور'],
      dtype='<U19')

In [9]:
most_imp_words = pd.Series(np.array(feature_names)[X.argmax(axis=1)].flatten())
most_imp_words.value_counts().head(20)

ول      534
اب      440
صلا     291
صل      270
اذ      242
مال     158
ام      152
مالك    150
حد      148
زكا     146
بيع     145
حج      138
شهاد     91
قتل      89
ارض      82
عتق      81
طلاق     77
رسول     77
زوج      77
غسل      75
dtype: int64

## Train model

In [10]:
nr_topics = 20

In [12]:
import lda
model = lda.LDA(n_topics=nr_topics, n_iter=500, random_state=1)
model.fit(X)

INFO:lda:n_documents: 10065
INFO:lda:vocab_size: 79401
INFO:lda:n_words: 25656190
INFO:lda:n_topics: 20
INFO:lda:n_iter: 500
/home/dafne/anaconda3/envs/adh/lib/python3.7/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -288552224
INFO:lda:<10> log likelihood: -253597784
INFO:lda:<20> log likelihood: -227908281
INFO:lda:<30> log likelihood: -224636409
INFO:lda:<40> log likelihood: -223296045
INFO:lda:<50> log likelihood: -222536561
INFO:lda:<60> log likelihood: -221984570
INFO:lda:<70> log likelihood: -221608036
INFO:lda:<80> log likelihood: -221322547
INFO:lda:<90> log likelihood: -221089555
INFO:lda:<100> log likelihood: -220896301
INFO:lda:<110> log likelihood: -220735744
INFO:lda:<120> log likelihood: -220560811
INFO:lda:<130> log likelihood:

In [13]:
# lda = LatentDirichletAllocation(n_components=nr_topics, random_state=0, max_iter=50)
# document_topics = lda.fit_transform(X)

In [14]:
document_topics = model.doc_topic_

## Save files

In [15]:
fp_out = '/media/sf_VBox_Shared/Arabic/Analyses/topicmodelling/'

In [16]:
df_document_topics = pd.DataFrame(document_topics, index=fnames)
df_document_topics.to_csv(os.path.join(fp_out, 'fiqh_light10_document_topics_{}.csv'.format(nr_topics)))

In [17]:
#lda.components_.shape
model.components_.shape

(20, 79401)

In [18]:
topic_words = pd.DataFrame(np.argsort(model.components_, axis=1)[:,-10:][::-1])

In [19]:
topic_words = topic_words.applymap(lambda l: feature_names[l])

In [20]:
topic_words.to_csv(os.path.join(fp_out, 'fiqh_light10_topics_{}.csv'.format(nr_topics)))

In [22]:
topic_words

,0,1,2,3,4,5,6,7,8,9
0,ورث,ام,اب,خمس,نصف,ثلاث,مال,عشر,وص,ثلث
1,رحم,وكذ,ظاهر,بخلاف,حنيف,يوسف,اب,ذكر,هذ,اذ
2,سجد,سلام,ركع,وقت,اول,صل,ول,اذ,امام,صلا
3,مال,مكاتب,ول,قيم,كتاب,ملك,حر,ام,ولد,عتق
4,طواف,محرم,حرم,احرام,عمر,اذ,صوم,يوم,ول,حج
5,اب,نفق,فل,اذ,عقد,مهر,ام,ول,نكاح,زوج
6,نجس,تيمم,مسح,دم,نجاس,صلا,وض,اذ,ول,غسل
7,هذ,ثمن,فل,قبض,ول,بيع,وكيل,اذ,حق,مال
8,ايض,وج,اذ,ظاهر,اول,ول,حيناذ,سلام,خلاف,عدم
9,نذر,اذ,فل,ول,هذ,كفار,يحنث,حنث,يم,حلف
